# Download and import necessary libraries


In [1]:
# download necessary libraries - only do this once 
# import sys
# !{sys.executable} -m pip install psycopg2 
# !{sys.executable} -m pip install pandas 
# !{sys.executable} -m pip install numpy

# Import necessary libraries
import os 
import glob # used to search for a specific file pattern 
import psycopg2 # for connecting to postgres database
import numpy as np # great substitute for array 
import pandas as pd # to be able to use dataframe which is utilized during transformation(staging) stage 
from table_queries import * # to call the sql queries we made
from dotenv import load_dotenv, find_dotenv # to access the secret keys we've hidden in a separate file 

load_dotenv(find_dotenv()) # grab values inside env file

True

# Connect to database
#### create a function that grabs the values inside your data


In [2]:
# connecting to database 
conn = psycopg2.connect(host="localhost", port="5432",database="sparkifydb",user="postgres",password=os.getenv("DB_PASSWORD"))
cur = conn.cursor() # used to execute sql queries

# create a function that grabs the values inside your data 
def get_files(filepath):
    all_files = [] 
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files


# Process Song Data
#### processing song_data dataset to be able to insert values at songs and artists dimensional tables 

In [3]:
# extraction 
song_data = get_files("data/song_data") 

filepath = song_data[0]

# loading - creating a dataframe(think of this as temporary storage) to transform and clean the data 
df = pd.read_json(filepath, lines=True)
df.head()

# transformation - we are supposed to do some transformations but the dataset is already good enough, so yeah thats it 


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### songs table 
#### performing etl to songs dataframe to be able to insert values at songs table 

In [4]:
# extraction & transformation - grabbing data and only getting the columns we want
songs_df = df[["song_id","title","artist_id","year","duration"]]

for i,row in songs_df.iterrows():
    # loading - putting data inside the sparkifydb inside songs table 
    cur.execute(songs_dimension_insert, list(row))
    conn.commit() # we pretty much need to do this at every query execution :(


### artists table
#### performing etl to artists dataframe to be able to insert values at artists table 

In [5]:
# extraction & transformation - grabbing data and only getting the columns we want
artists_df = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]]
artists_df.head()

for i,row in artists_df.iterrows():
    # loading - putting data inside the sparkifydb inside songs table 
    cur.execute(artists_dimension_insert, list(row))
    conn.commit() # we pretty much need to do this at every query execution :(
    print(songs_df)

              song_id             title           artist_id  year   duration
0  SOMZWCG12A8C13C480  I Didn't Mean To  ARD7TVE1187B99BFB1     0  218.93179
